In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=e56774d936de6578b0ecdde57b05c5927c8b24598f2da3e4bbb0e1f8d864b28f
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 12.5 GB  | Proc size: 95.7 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


In [ ]:
!pip install bert-tensorflow==1.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 6.4 MB/s 


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

tensorflow version :  1.15.2
tensorflow_hub version :  0.12.0



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!unzip /content/OpinRankDataset.zip
path = '/content/gdrive/MyDrive/MajorProject/Datasets/Hotel_Reviews.csv'

In [ ]:
dataset = pd.read_csv(path)

In [ ]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Additional_Number_of_Scoring,515738.0,498.081836,500.538467,1.000000,169.000000,341.000000,660.000000,2682.000000
Average_Score,515738.0,8.397487,0.548048,5.200000,8.100000,8.400000,8.800000,9.800000
Review_Total_Negative_Word_Counts,515738.0,18.539450,29.690831,0.000000,2.000000,9.000000,23.000000,408.000000
Total_Number_of_Reviews,515738.0,2743.743944,2317.464868,43.000000,1161.000000,2134.000000,3613.000000,16670.000000
Review_Total_Positive_Word_Counts,515738.0,17.776458,21.804185,0.000000,5.000000,11.000000,22.000000,395.000000
Total_Number_of_Reviews_Reviewer_Has_Given,515738.0,7.166001,11.040228,1.000000,1.000000,3.000000,8.000000,355.000000
Reviewer_Score,515738.0,8.395077,1.637856,2.500000,7.500000,8.800000,9.600000,10.000000
lat,512470.0,49.442439,3.466325,41.328376,48.214662,51.499981,51.516288,52.400181
lng,512470.0,2.823803,4.579425,-0.369758,-0.143372,0.010607,4.834443,16.429233


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [ ]:
positive_df = dataset[['Hotel_Name','Positive_Review','Reviewer_Score']]
positive_df.rename( columns={'Positive_Review':'Text_Review'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
negative_df = dataset[['Hotel_Name','Negative_Review','Reviewer_Score']]
negative_df.rename( columns={'Negative_Review':'Text_Review'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
positive_df.head(),negative_df.head()

(    Hotel_Name                                        Text_Review  \
 0  Hotel Arena   Only the park outside of the hotel was beauti...   
 1  Hotel Arena   No real complaints the hotel was great great ...   
 2  Hotel Arena   Location was good and staff were ok It is cut...   
 3  Hotel Arena   Great location in nice surroundings the bar a...   
 4  Hotel Arena    Amazing location and building Romantic setting    
 
    Reviewer_Score  
 0             2.9  
 1             7.5  
 2             7.1  
 3             3.8  
 4             6.7  ,
     Hotel_Name                                        Text_Review  \
 0  Hotel Arena   I am so angry that i made this post available...   
 1  Hotel Arena                                        No Negative   
 2  Hotel Arena   Rooms are nice but for elderly a bit difficul...   
 3  Hotel Arena   My room was dirty and I was afraid to walk ba...   
 4  Hotel Arena   You When I booked with your company on line y...   
 
    Reviewer_Score  
 0      

In [ ]:
final_df = pd.concat([positive_df,negative_df])

In [ ]:
final_df.head()

,Hotel_Name,Text_Review,Reviewer_Score
0,Hotel Arena,Only the park outside of the hotel was beauti...,2.9
1,Hotel Arena,No real complaints the hotel was great great ...,7.5
2,Hotel Arena,Location was good and staff were ok It is cut...,7.1
3,Hotel Arena,Great location in nice surroundings the bar a...,3.8
4,Hotel Arena,Amazing location and building Romantic setting,6.7


In [ ]:
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)



In [ ]:
# clean text data
cleaned_df = final_df
cleaned_df['Text_Review'] = final_df['Text_Review'].apply(lambda x: clean_text(x))
cleaned_df.head()

,Hotel_Name,Text_Review,Reviewer_Score
0,Hotel Arena,only the park outside of the hotel was beautiful,2.9
1,Hotel Arena,no real complaints the hotel was great great l...,7.5
2,Hotel Arena,location was good and staff were ok it is cute...,7.1
3,Hotel Arena,great location in nice surroundings the bar an...,3.8
4,Hotel Arena,amazing location and building romantic setting,6.7


In [ ]:
df = pd.DataFrame(cleaned_df.sort_values(by=['Hotel_Name']))
df.head()

,Hotel_Name,Text_Review,Reviewer_Score
33861,11 Cadogan Gardens,no negative,10.0
33757,11 Cadogan Gardens,the room was quite small although that was bec...,8.3
33756,11 Cadogan Gardens,no negative,9.6
33755,11 Cadogan Gardens,nothing,10.0
33754,11 Cadogan Gardens,room was very small and stuffy with very littl...,6.7


In [ ]:
df = df.reset_index(drop=True)
df.head()

,Hotel_Name,Text_Review,Reviewer_Score
0,11 Cadogan Gardens,no negative,10.0
1,11 Cadogan Gardens,the room was quite small although that was bec...,8.3
2,11 Cadogan Gardens,no negative,9.6
3,11 Cadogan Gardens,nothing,10.0
4,11 Cadogan Gardens,room was very small and stuffy with very littl...,6.7


In [ ]:
arr = df['Reviewer_Score'].unique()
arr

array([10. ,  8.3,  9.6,  6.7,  7.5,  7.9,  7.1,  9.2,  5. ,  8.8,  9.5,
        5.4,  4.5,  6.3,  4.2,  4.6,  8.5,  3.8,  5.8,  7. ,  2.9,  6. ,
        3.3,  9. ,  2.5,  8.1,  5.5,  8. ,  5.6,  3.5,  9.4,  6.5,  4. ,
        4.4,  3. ,  3.1,  6.9])

In [ ]:
for i in range(len(df['Reviewer_Score'])):
  if df['Reviewer_Score'][i] < 6.5:
    df['Reviewer_Score'][i] = 0.0
  elif df['Reviewer_Score'][i] < 8.5:
    df['Reviewer_Score'][i] = 1.0
  else:
    df['Reviewer_Score'][i] = 2.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df['Reviewer_Score'].value_counts()

2.0    587948
1.0    308160
0.0    135368
Name: Reviewer_Score, dtype: int64

In [ ]:
DATA_COLUMN = 'Text_Review'
LABEL_COLUMN = 'Reviewer_Score'
label_list = [0.0,1.0,2.0]

In [ ]:
df.shape

(1031476, 3)

In [ ]:
df.to_csv('cleaned_dataset.csv')

In [ ]:
df_train = df[:20000]
df_test = df[20000:]

In [ ]:
df_test.head()

,Hotel_Name,Text_Review,Reviewer_Score
20000,Amba Hotel Marble Arch,same food every day in the lounge,2.0
20001,Amba Hotel Marble Arch,the housekeeping are weak,0.0
20002,Amba Hotel Marble Arch,nothing,2.0
20003,Amba Hotel Marble Arch,nil,2.0
20004,Amba Hotel Marble Arch,the rooms are very small,0.0


In [ ]:
df_train['Reviewer_Score'].value_counts()

2.0    12070
1.0     5880
0.0     2050
Name: Reviewer_Score, dtype: int64

In [ ]:
train_InputExamples = df_train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = df_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
                               ...                        
19995    <bert.run_classifier.InputExample object at 0x...
19996    <bert.run_classifier.InputExample object at 0x...
19997    <bert.run_classifier.InputExample object at 0x...
19998    <bert.run_classifier.InputExample object at 0x...
19999    <bert.run_classifier.InputExample object at 0x...
Length: 20000, dtype: object

In [ ]:
#This is a path to an uncased (all lowercase) version of BERT

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return tokenization.FullTokenizer(
      vocab_file=vocab_file, 
      do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
print(tokenizer.tokenize(train_InputExamples.iloc[10].text_a))

['can', 'think', 'of', 'any', 'negative', '##s']


In [ ]:
#We'll set sequences to be at most 50 tokens long.
MAX_SEQ_LENGTH = 50

In [ ]:
#Convert our train and validation features to InputFeatures that BERT can understand
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list,MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list,MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 20000


INFO:tensorflow:Writing example 0 of 20000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1.0 (id = 1)


INFO:tensorflow:label: 1.0 (id = 1)


INFO:tensorflow:Writing example 10000 of 20000


INFO:tensorflow:Writing example 10000 of 20000


In [ ]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  no negative
------------------------------
Tokens :  ['no', 'negative']
------------------------------
Input IDs :  [101, 2053, 4997, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Input Masks :  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Segment IDs :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0

# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1

# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir='/content/bert-multiclass-sentiment',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/bert-multiclass-sentiment', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcde0d59310>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/bert-multiclass-sentiment', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcde0d59310>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
#Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

#Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
#Training the model
print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 1875 into /content/bert-multiclass-sentiment/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1875 into /content/bert-multiclass-sentiment/model.ckpt.


INFO:tensorflow:Loss for final step: 0.2972465.


INFO:tensorflow:Loss for final step: 0.2972465.


Training took time  0:13:53.020748


In [ ]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1875: eval_accuracy = 0.8296, false_negatives = 368.0, false_positives = 764.0, global_step = 1875, loss = 0.438158, true_negatives = 1286.0, true_positives = 17582.0


INFO:tensorflow:Saving dict for global step 1875: eval_accuracy = 0.8296, false_negatives = 368.0, false_positives = 764.0, global_step = 1875, loss = 0.438158, true_negatives = 1286.0, true_positives = 17582.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1875: /content/bert-multiclass-sentiment/model.ckpt-1875


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1875: /content/bert-multiclass-sentiment/model.ckpt-1875


{'eval_accuracy': 0.8296,
 'false_negatives': 368.0,
 'false_positives': 764.0,
 'global_step': 1875,
 'loss': 0.438158,
 'true_negatives': 1286.0,
 'true_positives': 17582.0}

In [ ]:
#To make predictions
def getPrediction(in_sentences):
  #A list to map the actual labels to the predictions
  labels = ["0.0", "1.0","2.0"]

  #Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
  #Creating input features for Test data
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
df_test.shape

(1011476, 3)

In [ ]:
testingdata = df_test[60000:65000]

In [ ]:
testingdata['Hotel_Name'].value_counts()

Best Western Madison Hotel                          752
Best Western Mornington Hotel Hyde Park             588
Best Western Hotel St George                        546
Best Western Hotel Montmartre Sacr Coeur            532
Best Western Hotel Ascot                            472
Best Western Maitrise Hotel Edgware Road            440
Best Western Hotel Pension Arenberg Wien Zentrum    398
Best Western Hotel City                             346
Best Western Le Jardin de Cluny                     240
Best Western Le 18 Paris                            172
Best Western Hotel Mirage                           166
Best Western Hotel Major                            146
Best Western Mercedes Arc de Triomphe               116
Best Western Hotel Astoria                           74
Best Western Ducs de Bourgogne                       12
Name: Hotel_Name, dtype: int64

In [ ]:
pred_sentences = list(testingdata['Text_Review'])
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/bert-multiclass-sentiment/model.ckpt-1875


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [ ]:
predictions_df = pd.DataFrame(predictions,columns= ['0','1','Pred_Rating','3'])

In [ ]:
predictions_df.drop(['0','1','3'],axis=1,inplace=True)

In [ ]:
predictions_df['Pred_Rating'].value_counts()

2    2527
1    1914
0     559
Name: Pred_Rating, dtype: int64

In [ ]:
predictions_df['Pred_Rating'] = predictions_df['Pred_Rating'].astype('float64')

In [ ]:
predictions_df['Pred_Rating'].dtypes

dtype('float64')

In [ ]:
for i in range(5000):
  testingdata['Pred_Rating'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
testingdata['Pred_Rating'][:] = predictions_df['Pred_Rating']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._set_values(indexer, value)


In [ ]:
testingdata['word_count'] = testingdata['Text_Review'].str.split().apply(len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
testingdata.tail()

,Hotel_Name,Text_Review,Reviewer_Score,Pred_Rating,word_count
84995,Best Western Mornington Hotel Hyde Park,the breakfast was great staff were friendly an...,1.0,1.0,12
84996,Best Western Mornington Hotel Hyde Park,room size was great,0.0,1.0,4
84997,Best Western Mornington Hotel Hyde Park,location and the price,1.0,2.0,4
84998,Best Western Mornington Hotel Hyde Park,the location was good and quite convenient upo...,0.0,2.0,21
84999,Best Western Mornington Hotel Hyde Park,proximity to city centre and underground,1.0,1.0,6


In [ ]:
hotellist = testingdata['Hotel_Name'].unique()
len(hotellist)

15

In [ ]:
startindex = 80000
endindex = 85000

In [ ]:
wordsumarr = np.zeros(len(hotellist))
wordsum = 0
j = 0
for i in range(startindex,endindex):
  if hotellist[j] == testingdata['Hotel_Name'][i]:
    wordsum += testingdata['word_count'][i]
  else:
    wordsumarr[j] = wordsum
    j+=1
    wordsum = 0

In [ ]:
wordsumarr

array([  491.,  6471.,  1123.,  5143.,  1823.,  2337., 10418.,  9150.,
        9613.,  2863.,  3446., 10852.,  8899.,  2220.,     0.])

In [ ]:
j = 0
sum = 0
hotelreviewsum = [[],[]]
hotelreviewsum[0][:] = hotellist
hotelreviewsum[1][:] = np.zeros(len(hotellist))

In [ ]:
print(np.transpose(hotelreviewsum))

[['Best Western Ducs de Bourgogne' '0.0']
 ['Best Western Hotel Ascot' '0.0']
 ['Best Western Hotel Astoria' '0.0']
 ['Best Western Hotel City' '0.0']
 ['Best Western Hotel Major' '0.0']
 ['Best Western Hotel Mirage' '0.0']
 ['Best Western Hotel Montmartre Sacr Coeur' '0.0']
 ['Best Western Hotel Pension Arenberg Wien Zentrum' '0.0']
 ['Best Western Hotel St George' '0.0']
 ['Best Western Le 18 Paris' '0.0']
 ['Best Western Le Jardin de Cluny' '0.0']
 ['Best Western Madison Hotel' '0.0']
 ['Best Western Maitrise Hotel Edgware Road' '0.0']
 ['Best Western Mercedes Arc de Triomphe' '0.0']
 ['Best Western Mornington Hotel Hyde Park' '0.0']]


In [ ]:

for i in range(startindex,endindex):
  if hotellist[j] == testingdata['Hotel_Name'][i]:
    sum += testingdata['Reviewer_Score'][i]*testingdata['word_count'][i]/wordsumarr[j]
  else:
    hotelreviewsum[1][j] = sum
    j+=1
    sum = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [ ]:
hotelreviewtranspose = np.transpose(hotelreviewsum)

In [ ]:
finalreviews = pd.DataFrame(hotelreviewtranspose,columns=['Hotel_name','Rating'])

In [ ]:
finalreviews.head()
finalreviews1 = finalreviews.drop('Hotel_name',axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
df_scaled = scaler.fit_transform(finalreviews1.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=[
 'Rating'])

In [ ]:
finalreviews['Rating'] = df_scaled*5

In [ ]:
finalreviews.round({'Rating':2})

,Hotel_name,Rating
0,Best Western Ducs de Bourgogne,2.16
1,Best Western Hotel Ascot,3.19
2,Best Western Hotel Astoria,5.00
3,Best Western Hotel City,4.43
4,Best Western Hotel Major,3.86
5,Best Western Hotel Mirage,4.16
6,Best Western Hotel Montmartre Sacr Coeur,2.61
7,Best Western Hotel Pension Arenberg Wien Zentrum,2.10
8,Best Western Hotel St George,3.79
9,Best Western Le 18 Paris,2.93


In [ ]:
#function not working yet
def getpredictions(testingdata,startindex,endindex):
  pred_sentences = list(testingdata['Text_Review'])
  predictions = getPrediction(pred_sentences)
  predictions_df = pd.DataFrame(predictions,columns= ['0','1','Pred_Rating','3'])
  predictions_df.drop(['0','1','3'],axis=1,inplace=True)
  predictions_df['Pred_Rating'].value_counts()
  predictions_df['Pred_Rating'] = predictions_df['Pred_Rating'].astype('float64')
  predictions_df['Pred_Rating'].dtypes
  
  for i in range(endindex-startindex):
    testingdata['Pred_Rating'] = 0

  testingdata['Pred_Rating'][:] = predictions_df['Pred_Rating']
  testingdata['word_count'] = testingdata['Text_Review'].str.split().apply(len)
  hotellist = testingdata['Hotel_Name'].unique()
  
  wordsumarr = np.zeros(len(hotellist))
  wordsum = 0
  j = 0
  for i in range(startindex,endindex):
   if hotellist[j] == testingdata['Hotel_Name'][i]:
    wordsum += testingdata['word_count'][i]
  else:
    wordsumarr[j] = wordsum
    j+=1
    wordsum = 0
  
  j = 0
  sum = 0
  hotelreviewsum = [[],[]]
  hotelreviewsum[0][:] = hotellist
  hotelreviewsum[1][:] = np.zeros(len(hotellist))
  
  for i in range(startindex,endindex):
    if hotellist[j] == testingdata['Hotel_Name'][i]:
      sum += testingdata['Pred_Rating'][i]*testingdata['word_count'][i]/wordsumarr[j]
    else:
      hotelreviewsum[1][j] = sum
      j+=1
      sum = 0
  
  hotelreviewtranspose = np.transpose(hotelreviewsum)
  finalreviews = pd.DataFrame(hotelreviewtranspose,columns=['Hotel_name','Rating'])
  finalreviews1 = finalreviews.drop('Hotel_name',axis=1)
  
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  # transform data
  df_scaled = scaler.fit_transform(finalreviews1.to_numpy())
  df_scaled = pd.DataFrame(df_scaled, columns=['Rating'])

  finalreviews['Rating'] = df_scaled*5
  
  return finalreviews.round({'Rating':2})

In [ ]:
completedpreds = getpredictions(testingdata,80000,85000)

INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from /content/bert-multiclass-sentiment/model.ckpt-1875


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [ ]:
completedpreds

,Hotel_name,Rating
0,Best Western Ducs de Bourgogne,5.0
1,Best Western Hotel Ascot,NaN
2,Best Western Hotel Astoria,NaN
3,Best Western Hotel City,NaN
4,Best Western Hotel Major,NaN
5,Best Western Hotel Mirage,NaN
6,Best Western Hotel Montmartre Sacr Coeur,NaN
7,Best Western Hotel Pension Arenberg Wien Zentrum,NaN
8,Best Western Hotel St George,NaN
9,Best Western Le 18 Paris,NaN


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
#!unzip /content/OpinRankDataset.zip
path = r'/content/gdrive/MyDrive/IITData/new-delhi'

In [ ]:
def fileNameCreator(path):
  import glob
  filenames = glob.glob(path + "/*")
  return filenames

In [ ]:
filenames = fileNameCreator(path)

In [ ]:
print(len('/content/gdrive/MyDrive/IITData/new-delhi/india_new delhi_')),filenames[0]

58


IndexError: ignored

In [ ]:
def readFile(filename):
  allphrases = []
  alldates = []
  for i in range(len(filename)):
        fileObj = open(filename[i], "r",encoding='utf-8', errors='ignore') #opens the file in read mode
        phrases = fileObj.read() #puts the file into an array
        alldates.append(phrases[:11])
        allphrases.append(phrases)
        fileObj.close()
  return allphrases,alldates

In [ ]:
review_data, review_dates = readFile(filenames)
#review_data = str(review_data).split('/n')
alldata = pd.DataFrame(review_data)
len(alldata)

0

In [ ]:
allhotelnames = []
alldates = []
for i in range(130):
  allhotelnames.append(filenames[i][58:].replace('_',' '))
len(allhotelnames)

In [ ]:
allhotelnames = pd.DataFrame(allhotelnames)
totaldata = pd.concat([allhotelnames, alldata],axis=1, ignore_index=True)
hotelno=1

In [ ]:
def datacompilerfunc(totaldata):
  completedata = pd.DataFrame()
  for hotelno in range(len(allhotelnames)):
    compstr = totaldata[1][hotelno]
    comparr = compstr.splitlines(False)
    datearr = []

    filenamearr = []
    for i in range(len(comparr)):
      filenamearr.append(totaldata[0][hotelno])
    k = 0
    for i in range(len(comparr)):
      if(comparr[i][0]=='\t'):
        datearr.append('')
        comparr[i] = comparr[i][1:].replace('\t',' ')
      else:
        for j in range(12):
          if comparr[i][j] == '\t':
            break
        datearr.append(comparr[i][0:j])
        comparr[i] = comparr[i][j+1:].replace('\t',' ')

    reviewdf = pd.DataFrame(comparr, columns=['Review'])
    datesconv = pd.to_datetime(datearr, errors='ignore')
    datesdf = pd.DataFrame(datesconv, columns=['Date'])
    hotelnamedf = pd.DataFrame(filenamearr, columns = ["Hotel_name"])
    allrefineddata = pd.concat([hotelnamedf,datesdf,reviewdf], axis = 1)
    completedata = pd.concat([completedata,allrefineddata], axis = 0)
  
  return completedata

In [ ]:
dataset = datacompilerfunc(totaldata)

In [ ]:
dataset.drop(['Date'],axis=1,inplace=True)

In [ ]:
dataset.rename(columns={'Hotel_name':'Hotel_Name', 'Review':'Text_Review'},inplace=True)

In [ ]:
dataset.reset_index(inplace= True)
dataset.head()

In [ ]:
pred_sentences = list(dataset['Text_Review'])
predictions = getPrediction(pred_sentences)
predictions_df = pd.DataFrame(predictions,columns= ['0','1','Pred_Rating','3'])
predictions_df.drop(['0','1','3'],axis=1,inplace=True)
predictions_df['Pred_Rating'].value_counts()
predictions_df['Pred_Rating'] = predictions_df['Pred_Rating'].astype('float64')
predictions_df['Pred_Rating'].dtypes

In [ ]:
for i in range(4917):
  dataset['Pred_Rating'] = 0

dataset['Pred_Rating'][:] = predictions_df['Pred_Rating']
dataset['word_count'] = dataset['Text_Review'].str.split().apply(len)
hotellist = dataset['Hotel_Name'].unique()
  
wordsumarr = np.zeros(len(hotellist))
wordsum = 0
j = 0
for i in range(4917):
 if hotellist[j] == dataset['Hotel_Name'][i]:
  wordsum += dataset['word_count'][i]
 else:
  wordsumarr[j] = wordsum
  j+=1
  wordsum = 0
  
j = 0
sum = 0
hotelreviewsum = [[],[]]
hotelreviewsum[0][:] = hotellist
hotelreviewsum[1][:] = np.zeros(len(hotellist))

In [ ]:
j

In [ ]:
for i in range(4917):
  if hotellist[j] == dataset['Hotel_Name'][i]:
    sum += dataset['Pred_Rating'][i]*dataset['word_count'][i]/wordsumarr[j]
  else:
    hotelreviewsum[1][j] = sum
    j+=1
    sum = 0
  
hotelreviewtranspose = np.transpose(hotelreviewsum)
finalreviews = pd.DataFrame(hotelreviewtranspose,columns=['Hotel_name','Rating'])
finalreviews1 = finalreviews.drop('Hotel_name',axis=1)
  
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
df_scaled = scaler.fit_transform(finalreviews1.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=['Rating'])

finalreviews['Rating'] = df_scaled*5
  
indianreviews = finalreviews.round({'Rating':2})

In [ ]:
indianreviews.to_csv('indianrankings.csv')

In [ ]:
from google.colab import files
files.download('indianrankings.csv') 